
#Introduction
Our three metrics that we used to evaluate are:
- Crime
- Education
- Green spaces

The way we evaluate these spaces 

In [ ]:
#read datatsets
arrest_data = pd.read_csv("arrest_data.csv")
parks_data = pd.read_csv("parks_data.csv")
income_data = pd.read_csv("income_data.csv")

In [ ]:
#Arrest Analysis (Nick)
arrest_count = arrest_data['INCIDENTNEIGHBORHOOD'].value_counts().reset_index()
arrest_count.columns = ['Neighborhood', 'Arrests']

# Rank Arrest Data
combined_data = arrest_count.copy()
combined_data['ArrestsRank'] = combined_data['Arrests'].rank(ascending=True)


In [ ]:
# Parks Analysis (Avneet)
parks_count = parks_data['neighborhood'].value_counts().reset_index()
parks_count.columns = ['Neighborhood', 'Parks']

# Rank Parks Data
combined_data = parks_count.copy()
combined_data['ParksRank'] = combined_data['Parks'].rank(ascending=False)


In [ ]:
# Process Income Data (Evan)
income_data = income_data[['Community', 'median_household']]
income_data = income_data.rename(columns={'Community': 'Neighborhood', 'median_household': 'Median Income'})
income_data = income_data.replace(',','', regex=True)
income_data['Median Income'] = income_data['Median Income'].astype(float)

# Rank Income Data
combined_data = income_data.copy()
combined_data['IncomeRank'] = combined_data['Median Income'].rank(ascending=False)


In [ ]:
#dipslay the data (in a bar graph that depicts the lowest rank (lowest bar) as the best neighborhood
import matplotlib.pyplot as plt

# Calculate total rank and sort the data by it
plot_data = combined_data[['Neighborhood', 'ArrestsRank', 'ParksRank', 'IncomeRank']].set_index('Neighborhood')
plot_data['TotalRank'] = plot_data['ArrestsRank'] + plot_data['ParksRank'] + plot_data['IncomeRank']
plot_data = plot_data.sort_values('TotalRank', ascending=True)

# Create the stacked bar plot
fig, ax = plt.subplots(figsize=(20, 10))
plot_data[['ArrestsRank', 'ParksRank', 'IncomeRank']].plot(kind='bar', stacked=True, ax=ax)
ax.set_title("Neighborhood Rankings for Arrests, Parks, and Income (Sorted by Total Rank)")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("Rank")

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# adjusted code (using weights)
# Import Libraries
import pandas as pd
import numpy as np

# Read Datasets
arrest_data = pd.read_csv("arrest_data.csv")
parks_data = pd.read_csv("parks_data.csv")
income_data = pd.read_csv("income_data.csv")

# Process Arrest Data
arrest_count = arrest_data['INCIDENTNEIGHBORHOOD'].value_counts().reset_index()
arrest_count.columns = ['Neighborhood', 'Arrests']

# Process Parks Data
parks_count = parks_data['neighborhood'].value_counts().reset_index()
parks_count.columns = ['Neighborhood', 'Parks']

# Process Income Data
income_data = income_data[['Community', 'median_household']]
income_data = income_data.rename(columns={'Community': 'Neighborhood', 'median_household': 'Median Income'})

# Remove commas from the 'Median Income' column
income_data['Median Income'] = income_data['Median Income'].str.replace(',', '')

# Merge All Data
combined_data = arrest_count.merge(parks_count, on='Neighborhood', how='outer').merge(income_data, on='Neighborhood', how='outer')

# Convert 'Median Income' to a numeric type
combined_data['Median Income'] = pd.to_numeric(combined_data['Median Income'], errors='coerce')

# Fill missing values with 0
combined_data = combined_data.fillna(0)

# Drop rows where 'Arrests', 'Parks', and 'Median Income' are all zero
combined_data = combined_data[(combined_data['Arrests'] != 0) | (combined_data['Parks'] != 0) | (combined_data['Median Income'] > 0)]

# Rank Each Metric
combined_data['ArrestsRank'] = combined_data['Arrests'].rank(ascending=False)
combined_data['ParksRank'] = combined_data['Parks'].rank(ascending=False)
combined_data['IncomeRank'] = combined_data['Median Income'].rank(ascending=False)

# Assign weights to each metric
arrests_weight = 5
parks_weight = 3
income_weight = 2

# Calculate Weighted Combined Score
combined_data['CombinedScore'] = (combined_data['ArrestsRank'] * arrests_weight) + (combined_data['ParksRank'] * parks_weight) + (combined_data['IncomeRank'] * income_weight)

# Find the Best Neighborhood
best_neighborhood = combined_data.loc[combined_data['CombinedScore'].idxmin(), 'Neighborhood']
print(f"The best neighborhood in Pittsburgh is {best_neighborhood}.")

combined_data = combined_data.sort_values('CombinedScore', ascending=False)
combined_data.tail(10)


In [ ]:
#dipslay the data for (weighted options)
# Calculate total rank and sort the data by it
plot_data = combined_data[['Neighborhood', 'ArrestsRank', 'ParksRank', 'IncomeRank']].set_index('Neighborhood')
plot_data['TotalRank'] = plot_data['ArrestsRank'] + plot_data['ParksRank'] + plot_data['IncomeRank']
plot_data = plot_data.sort_values('TotalRank', ascending=True)

# Create the stacked bar plot
fig, ax = plt.subplots(figsize=(20, 10))
plot_data[['ArrestsRank', 'ParksRank', 'IncomeRank']].plot(kind='bar', stacked=True, ax=ax)
ax.set_title("Neighborhood Rankings for Arrests, Parks, and Income (Sorted by Total Rank)")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("Rank")

# Display the plot
plt.tight_layout()
plt.show()